In [14]:
import numpy as np

##### Summarize Date Df

In [15]:
# Summarize df with Date
def summarize_df(df):
    print(df.shape[0])
    print(min(df['Date']))
    print(max(df['Date']))
    df

#### Preprocessing

##### Fill empty weeks of date Df

In [16]:
def Fix_Data(df):
    my_date = datetime.now() # if date is 01/01/2018
    year, week_num, day_of_week = my_date.isocalendar()
    d = Week(year, week_num-1).monday()
    rng = pd.date_range(df['Date'].min(), d,freq='7D')
    df = df.set_index('Date').reindex(rng, fill_value=0).reset_index() 
    df.rename(columns={'index':'Date'},inplace=True)
    return df

##### Clean outliers

In [17]:
def cap_outliers(df, max_outlier_value):
    df.loc[df['Net Order Value']>max_outlier_value,'Net Order Value'] = max_outlier_value
    return df

##### Adjust Baseline

In [18]:
def adjust_baseline(df, change_date, end_date):
    # Calculate baseline avg difference between TS before change_date vs TS between change_date and end_date
    diff_high_low = df.loc[(change_date < df['Date']) & (df['Date'] <= end_date),'Net Order Value'].mean()\
        -df.loc[df['Date'] <= change_date, 'Net Order Value'].mean()
    # Adjust lower baseline with the above avg difference
    df.loc[df['Date'] <= change_date,'Net Order Value'] += diff_high_low
    return df

#### Plots

In [40]:
def plot_clean_y(df,train_df,max_outlier_value,y_margin):
    fig, ax = plt.subplots(figsize=(20,10))
    plt.plot(df['Date'], df['Net Order Value'], c='c',label='Y Original')
    plt.plot(train_df['Date'], train_df['Net Order Value'], c='b',label='Y')
    plt.legend(loc="upper right")
    plt.axis([min(train_df['Date']), max(train_df['Date']), 0, max_outlier_value+y_margin]);
    plt.show()

def plot_y_trend(train_df,t,y_min,y_max):
    fig, ax = plt.subplots(figsize=(20,10))
    plt.plot(train_df['Date'],t,color='b',label='Trend')
    plt.plot(train_df['Date'],train_df['Net Order Value'],color='g',label='Y')
    plt.legend(loc="upper right")
    ax.set_ylim([y_min,y_max])
    plt.show()

def plot_y_trend_ext(train_df,exo_col_name,exo_pretty_name,y_min,y_max,y_min_exo,y_max_exo):
    fig, ax = plt.subplots(figsize=(20,10))
    ax2 = ax.twinx()
    # Net Order Value
    ax.plot(train_df.index,train_df['Net Order Value'],color='g',label='Y')
    # External data/GDP
    ax2.plot(train_df.index,train_df[exo_col_name],color='c',label=exo_pretty_name)
    # Trend
    ax.plot(train_df.dropna().index,Y,color='b',label='Trend')
    plt.legend(loc="upper right")
    ax.set_ylim([y_min,y_max])
    ax2.set_ylim([y_min_exo,y_max_exo])
    plt.show()
    
def plot_y_pred_trend_ext(train_df,exo_col_name,X,Y,X_F,y_min,y_max,y_min_exo,y_max_exo):
    fig, ax = plt.subplots(figsize=(20,10))
    ax2 = ax.twinx()
    # External Data/GDP
    ax2.plot(train_df[exo_col_name].dropna().index,train_df[exo_col_name].dropna(),color='m',label='External data (full)')
    ax2.plot(train_df.dropna().index,X,color='c',label='External data (train for T fit)')
    # Trend
    ax.plot(train_df.dropna().index,Y,color='b',label='Trend (train for T fit)')
    # Reg predicted Trend
    ax.plot(train_df[exo_col_name].dropna().index,reg.predict(X_F),color='g',label='T Pred')
    plt.legend(loc="upper right")
    ax.set_ylim([y_min,y_max])
    ax2.set_ylim([y_min_exo,y_max_exo])
    #ax.set_ylim([4000000,9000000])
    #ax2.set_ylim([100,200])
    plt.show()
    
def plot_y_t_s(train_df,trend_col_name,seasonality_col_name):
    fig, ax = plt.subplots(figsize=(20,10))
    plt.plot(train_df.index,train_df['Net Order Value'],color='g',label='Y')
    plt.plot(train_df.index,train_df[trend_col_name],color='b',label='T')
    plt.plot(train_df.index,train_df[trend_col_name]+train_df[seasonality_col_name],color='m',label='T+S')
    plt.legend(loc="upper right")
    plt.show()
    
def plot_y_t_s_with_pred(train_df,trend_col_name,seasonality_col_name,pred_trend_col_name):
    fig, ax = plt.subplots(figsize=(20,10))
    plt.plot(train_df.index,train_df['Net Order Value'],color='g',label='Y')
    plt.plot(train_df.index,train_df[trend_col_name],color='b',label='T')
    plt.plot(train_df.index,train_df[pred_trend_col_name],color='c',label='T Pred')

    plt.plot(train_df.index,train_df[trend_col_name]+train_df[seasonality_col_name],color='m',label='T+S')
    plt.plot(train_df.index,train_df[pred_trend_col_name]+train_df[seasonality_col_name],color='r',label='T Pred + S')
    plt.legend(loc="upper right")
    plt.show()
    
def plot_r(train_df,r_col_name):
    fig, ax = plt.subplots(figsize=(20,10))
    plt.plot(train_df.index,train_df[r_col_name],color='y',label='R')
    plt.legend(loc="upper right")
    plt.show()
    
def plot_acf_pacf_r(r,lags):
    fig,ax = plt.subplots(2,1,figsize=(20,10))
    fig = plot_acf(r.dropna(), lags=lags, ax=ax[0])
    fig = plot_pacf(r.dropna(), lags=lags, ax=ax[1])
    plt.show()
    
def plot_final(train_df,trend_col_name,seasonality_col_name,r_col_name,trend_pred_col_name,y_pred_col_name):
    fig, ax = plt.subplots(figsize=(20,10))
    plt.plot(train_df.index,train_df['Net Order Value'],color='g',label='Y')
    plt.plot(train_df.index,train_df[trend_col_name],color='b',label='T')
    plt.plot(train_df.index,train_df[trend_col_name]+train_df[seasonality_col_name],color='m',label='T+S')
    # Seasonality
    plt.plot(train_df.index,train_df[seasonality_col_name],color='m',label='S')
    # Predicted Trend
    plt.plot(train_df.index,train_df[trend_pred_col_name],color='y',label='T Pred')
    plt.plot(train_df.index,train_df[trend_pred_col_name]+train_df[seasonality_col_name],color='k',label='T Pred + S')
    # Predicted Y on Validation part
    plt.plot(train_df[train_df['Predicted R Classification']=='test'].index,
             train_df[train_df['Predicted R Classification']=='test'][y_pred_col_name],color='c',label='Y Pred (val)')
    # Predicted Y on Future part
    plt.plot(train_df[train_df['Predicted R Classification']=='forecast'].index,
             train_df[train_df['Predicted R Classification']=='forecast'][y_pred_col_name],color='r',label='Y Pred (future)')
    plt.legend(loc="upper right")
    plt.show()

#### Trend

##### Calculate Trend

In [23]:
def calculate_trend(df,ts_seasonality, center=False):
    t = df.iloc[:,1].rolling(window=ts_seasonality,center=center).mean()
    return t

##### Combine External/GDP data with Y

In [21]:
def combine_ext_data(train_df,ext_data,days_to_shift=1):
    # Add Exo regressors (GDP) to train df
    train_df = train_df.set_index('Date')
    ext_data['DATE'] = pd.to_datetime(ext_data['DATE'])
    # Optional - Align dates of Industry GDP with Trend
    if days_to_shift is not None:
        ext_data = ext_data.set_index('DATE').shift(days_to_shift,freq='D')

    # Combine Train Df with GDP
    train_df = train_df.combine_first(ext_data)
    return train_df

##### Create subsets for Trend Fit

In [26]:
def subsets_to_fit(train_df,exo_col_name,trend_col_name,val_size_perc):
    # Create X set (exo regressor)
    X = train_df.dropna()[exo_col_name].values
    train_size = int(len(X) * (1-val_size_perc))
    X_train = X[:train_size].reshape(-1,1)
    # Create Y set (Trend to fit)
    Y_train = train_df.dropna()[trend_col_name].values[:train_size].reshape(-1,1)
    
    return X_train,Y_train

##### Trend Regression to predict future Trend

In [29]:
def predict_trend(train_df,exo_col_name,pred_trend_col_name):
    X_F = train_df[exo_col_name].dropna().values.reshape(-1,1)
    print(X_F.shape)
    print(reg.predict(X_F).shape)
    ## Add Predicted Trend to df
    t_pred = reg.predict(X_F)
    len_pred = t_pred.shape[0]
    train_df['Predicted Trend'] = np.nan
    train_df['Predicted Trend'][-len_pred:] = t_pred.ravel()
    return X_F,train_df

#### Seasonality

In [33]:
def calc_seasonality(train_df,col_index=4,window=10,center=True):
    s = train_df.iloc[:,col_index].rolling(window=10,center=True).mean()
    return s

#### Residuals

In [38]:
def create_r_df(train_df,columns_to_drop,col_to_rename):
    r_df = train_df.copy()
    r_df = r_df.drop(columns=columns_to_drop)
    r_df = r_df.reset_index()
    r_df = r_df.rename(columns=col_to_rename)
    return r_df

def add_r(train_df,results_df_r,r_col_name):
    results_df_r_idx = results_df_r.set_index('Date')
    train_df[r_col_name] = np.nan
    train_df[r_col_name] = results_df_r_idx['Predicted Net Order Value']
    train_df[r_class_col_name] = results_df_r_idx['Classification']
    return train_df

##### Fit ARIMA model + Predict future values

In [22]:
# Fit ARIMA on input df (optional input and future exo regr) and predict validation + future values
# Or use param fitted model (optional input and future exo regr) to predict validation + future values
# Plot input and output (val+future) predictions
def get_results_with_val(df,exo,p,d,q,P,D,Q,s, model, n_predictions=18,value_col_name,val_size_perc):

    X = df[value_col_name].values
    Y = df['Date'].values
    
    train_size = int(len(X) * (1-val_size_perc))
    train, test = X[:train_size], X[train_size:len(X)]
    week = Y[train_size:len(X)]
    
    exo_past,exo_future = None,None
    if exo is not None:
        exo_past,exo_future = exo[:len(X)], exo[len(X):len(exo)]
    
    print('Checking model for fit...')
    if model is None:
        print('No input model, will fit SARIMA'+str(p)+str(d)+str(q)+str(P)+str(D)+str(Q)+str(s))
        print('Starting Arima fit...')
        smodel = pm.arima.ARIMA(order=[p,d,q]
                                #,
                                #seasonal_order=[P,D,Q,s]
                                ,method='lbfgs'
                            ,maxiter=50,suppress_warnings=True)\
                             .fit(df[value_col_name].values,exo_past)
        print('Finished Arima fit.')
    else:
        print('Existing model, will use it')
        smodel = model
    

    history = [x for x in train]
    predictions = list()
    for t in range(len(test)):
        model = SARIMAX(history, order=smodel.order, seasonal_order=smodel.seasonal_order,enforce_stationarity=False)
        model_fit = model.fit(disp=0)
        output = model_fit.forecast()
        if output[0]<0:
           yhat=0
        else: yhat =output[0]
        predictions.append(yhat)
        obs = test[t]
        history.append(obs)
        print('predicted=%f, expected=%f' % (yhat, obs))
    error = mean_squared_error(test, predictions)
    print('Test MSE: %.3f' % error)
    

    # plot
    #pyplot.plot(test)
    #pyplot.plot(predictions, color='red')
    #pyplot.show()
    
    # Train set (train data)
    data=pd.DataFrame()
    data['Date']=Y[0:train_size]
    data['Predicted Net Order Value']=NaN
    data['Actual Net Order Value']=X[0:train_size]
    data['Classification']='train'
    
    # Validation set (val data with predictions)
    Tested=pd.DataFrame()
    Tested['Date'] = week
    Tested['Predicted Net Order Value']=predictions
    Tested['Actual Net Order Value']=test
    Tested['Classification'] = 'test'
    Tested['Predicted Net Order Value'] = Tested['Predicted Net Order Value'].astype(float) # TEST
    Tested['Date'] =  pd.to_datetime(Tested['Date'])

    # Forecast set (out-of-sample predictions)
    print('Starting to predict future values...')
    n_periods=n_predictions
    fitted, confint = smodel.predict(n_periods=n_periods, return_conf_int=True, exogenous=exo_future)
    print('Finished to predict future values.')
    rng = pd.date_range(df['Date'].max(), periods=n_periods, freq='7D')
    forecast = pd.DataFrame({ 'Date': rng, 'Predicted Net Order Value': fitted, 'Actual Net Order Value':NaN, 'Classification': 'forecast', 'Conf_lower':confint[:, 0], 'Conf_Upper':confint[:, 1] }) 
    forecast = forecast.drop(forecast.index[0])

    # All sets combined
    results = data.append(Tested, ignore_index=True)
    results = results.append(forecast, ignore_index=True)
    results['Date'] =  pd.to_datetime(results['Date'])
    
    # Plot all sets (input and predictions)
    fig, ax = plt.subplots(figsize=(20,10))
    results['Date'] = results['Date'].astype(str)
    plt.plot(results['Date'], results['Predicted Net Order Value'], c='b')
    plt.plot(results['Date'], results['Actual Net Order Value'], c='r')
    plt.fill_between(results['Date'], 
                     results['Conf_lower'], 
                     results['Conf_Upper'], 
                     color='k', alpha=.15)
    for i, tick in enumerate(ax.get_xticklabels()):
        tick.set_rotation(45)
        tick.set_visible(False)
        if i % 3 == 0:
            tick.set_visible(True)
    plt.show()
    
    # Reformat Dates to Date type
    results['Date'] =  pd.to_datetime(results['Date'])

    return smodel,results

#### Y Predictions

In [39]:
def calc_y_pred(train_df,y_pred_col_name,trend_pred_col_name,seasonality_col_name,r_class_col_name):
    train_df[y_pred_col_name] = np.nan
    # Validation Y values
    train_df.loc[train_df[r_class_col_name]=='test',y_pred_col_name] = train_df[trend_pred_col_name]\
                                + train_df[seasonality_col_name]\
                                + train_df['Predicted R']
    # Future Y values
    train_df.loc[train_df[r_class_col_name]=='forecast',y_pred_col_name] = train_df[trend_pred_col_name]\
                                + train_df[seasonality_col_name]\
                                + train_df['Predicted R']